In [2]:
import os
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers



In [3]:
img_height, img_width = 128, 128
batch_size = 32

train_ds = tf.keras.utils.image_dataset_from_directory(
    "dataset/train",
    image_size=(img_height, img_width),
    batch_size=batch_size,
    label_mode='categorical'
)
val_ds = tf.keras.utils.image_dataset_from_directory(
    "dataset/val",
    image_size=(img_height, img_width),
    batch_size=batch_size,
    label_mode='categorical'
)   
test_ds = tf.keras.utils.image_dataset_from_directory(
    "dataset/test",
    image_size=(img_height, img_width),
    batch_size=batch_size,
    label_mode='categorical'
)

Found 99 files belonging to 6 classes.
Found 20 files belonging to 6 classes.
Found 21 files belonging to 6 classes.


In [4]:
num_classes = len(train_ds.class_names)

In [5]:
normalization_layer = layers.Rescaling(1./255)

train_ds = train_ds.map(lambda x, y: (normalization_layer(x), y))
val_ds = val_ds.map(lambda x, y: (normalization_layer(x), y))
test_ds = test_ds.map(lambda x, y: (normalization_layer(x), y))


In [6]:
data_augmentation = keras.Sequential([
    layers.RandomFlip("horizontal_and_vertical"),
    layers.RandomRotation(0.2),
    layers.RandomZoom(0.1),
])
train_ds = train_ds.map(lambda x, y: (data_augmentation(x, training=True), y))

In [7]:
def mobile_net_v2_model(num_classes):   
    base_model = keras.applications.MobileNetV2(
        input_shape=(128, 128, 3),
        include_top=False,
        weights='imagenet'
    )

    base_model.trainable = False  # freeze pretrained layers

    model = keras.Sequential([
        base_model,
        layers.GlobalAveragePooling2D(),
        layers.Dense(128, activation='relu'),
        layers.Dense(num_classes, activation='softmax')
    ])

    model.compile(
        optimizer='adam',
        loss='categorical_crossentropy',
        metrics=['accuracy']
    )
    return model

In [8]:
mnv2_model = mobile_net_v2_model(num_classes)

history = mnv2_model.fit(
    train_ds,
    validation_data=val_ds,
    epochs=20
)


Epoch 1/20
4/4 ━━━━━━━━━━━━━━━━━━━━ 6s 730ms/step - accuracy: 0.2626 - loss: 1.9170 - val_accuracy: 0.2500 - val_loss: 1.6935
Epoch 2/20
4/4 ━━━━━━━━━━━━━━━━━━━━ 2s 451ms/step - accuracy: 0.4848 - loss: 1.3772 - val_accuracy: 0.3000 - val_loss: 1.7783
Epoch 3/20
4/4 ━━━━━━━━━━━━━━━━━━━━ 2s 468ms/step - accuracy: 0.5859 - loss: 1.1683 - val_accuracy: 0.3000 - val_loss: 1.7842
Epoch 4/20
4/4 ━━━━━━━━━━━━━━━━━━━━ 3s 502ms/step - accuracy: 0.6162 - loss: 1.0081 - val_accuracy: 0.4000 - val_loss: 1.5598
Epoch 5/20
4/4 ━━━━━━━━━━━━━━━━━━━━ 2s 461ms/step - accuracy: 0.6465 - loss: 0.9323 - val_accuracy: 0.4500 - val_loss: 1.4396
Epoch 6/20
4/4 ━━━━━━━━━━━━━━━━━━━━ 2s 453ms/step - accuracy: 0.7172 - loss: 0.8295 - val_accuracy: 0.3500 - val_loss: 1.5628
Epoch 7/20
4/4 ━━━━━━━━━━━━━━━━━━━━ 2s 467ms/step - accuracy: 0.7475 - loss: 0.8038 - val_accuracy: 0.4500 - val_loss: 1.2973
Epoch 8/20
4/4 ━━━━━━━━━━━━━━━━━━━━ 2s 480ms/step - accuracy: 0.8182 - loss: 0.5769 - val_accuracy: 0.5000 - val_loss:

In [9]:
mnv2_model.save('food_classification_model.keras')

In [16]:
scores = mnv2_model.evaluate(test_ds)
print("Test loss:", scores[0])
print("Test accuracy:", scores[1])


1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 514ms/step - accuracy: 0.3333 - loss: 1.9104
Test loss: 1.9104143381118774
Test accuracy: 0.3333333432674408
